<a href="https://colab.research.google.com/github/RebelBeast/Videogames/blob/main/Kopie_von_backloggd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import regex as re
from lxml import etree as et
from itertools import repeat
import csv
import random
import time
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')

In [ ]:
header_list = ["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML,                           like Gecko) Chrome/103.0.5060.66 Safari/537.36",
              "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:53.0) Gecko/20100101 Firefox/53.0",
              "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393"]

base_url = 'https://www.backloggd.com/games/lib/popular?page='
games_list = []
d = 0

In [ ]:
game_links = []

for page_no in tqdm_notebook(range(200)):
    page_url = base_url + str(page_no)
    #print(page_url)
    user_agent = random.choice(header_list)
    header = {"User-Agent": user_agent}
    webpage = requests.get(page_url, headers = header)
    if webpage.status_code == 200:
        soup1 = BeautifulSoup(webpage.content, 'html.parser')
        soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')
        g = soup2.find('div', {'class' : 'row show-release toggle-fade'})
        games = g.find_all('div', {'class' : 'col-2 my-2 px-1 px-md-2'})
        game_ref = [game.find('a').get('href') for game in games]
        game_links.extend(['https://www.backloggd.com' + ref for ref in game_ref])

  0%|          | 0/200 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
print(len(game_links))

2304


In [ ]:
cols = ['Title', 'Release Date', 'Team', 'Rating', 'Times Listed', 'Number of Reviews', 'Genres', 'Summary', 'Reviews', 'Plays', 'Playing', 'Backlogs', 'Wishlist']
backloggd = pd.DataFrame(columns = cols)

In [ ]:
for link in tqdm_notebook(game_links):
    user_agent = random.choice(header_list)
    header = {"User-Agent": user_agent}
    webpage = requests.get(link, headers = header)
    if webpage.status_code == 200:
        soup1 = BeautifulSoup(webpage.content, 'html.parser')
        soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')
        
        title = soup2.find('div', {'class' : 'col-auto pr-1'}).get_text().strip()
        
        release_date = ' '.join(soup2.find('div', {'class' : 'col-auto mt-auto pr-0'}).get_text().strip().split()[-3:])

        try:
            teams = soup2.find('div', {'class' : 'col-auto pl-lg-1 sub-title'})
            teams = teams.find_all('a')
            teams = [i.get_text().strip() for i in teams]
        except:
            teams = np.nan
            #print(teams)
        
        try:
            rating = float(soup2.find(id = 'score').get_text().strip()[-3:])
        except:
            rating = np.nan

        table = soup2.find_all('div', {'class' : 'col-12 mb-1'})#.get_text().strip()
        feats = [f.get_text().strip().split('\n')[0].strip() for f in table]
        results = [r.get_text().strip().split('\n')[-1].strip() for r in table]
        #print(feats,results)

        # dicted = {}
        # for i in range(len(feats)):
        #     dicted[feats[i]] = results[i]
        
        nlists = soup2.find('p', {'class' : 'game-page-sidecard'}).get_text().strip().split()[0]
        
        nreviews = soup2.find('p', {'class' : 'game-page-sidecard'}).get_text().strip().split()[1]

        genres = soup2.find_all('p',{'class' : 'genre-tag'})
        genres = [genre.get_text().strip() for genre in genres]

        summary = soup2.find(id = 'collapseSummary').get_text().strip()

        review_section = soup2.find(id = 'game-reviews-section')
        reviews = review_section.find_all('div', {'class' : 'row pt-2 pb-1 review-card'})
        reviews = [r.find('div', {'class' : 'formatted-text'}).get_text().strip() for r in reviews]

        row = [title, release_date, teams, rating, nlists, nreviews, genres, summary, reviews]
        row.extend(results)
        backloggd.loc[len(backloggd.index)] = row

  0%|          | 0/2304 [00:00<?, ?it/s]

In [ ]:
backloggd.tail()

,Title,Release Date,Team,Rating,Times Listed,Number of Reviews,Genres,Summary,Reviews,Plays,Playing,Backlogs,Wishlist
115,MediEvil,"Oct 09, 1998","[Sony Computer Entertainment Europe, SCE Cambr...",3.5,240,Lists,"[Adventure, Brawler, Platform, Puzzle]",The game takes place across a variety of level...,"[When I originally played this as a youngin', ...",1.6K,16,489,269
116,Five Nights at Freddy's: Sister Location,"Oct 07, 2016","[Clickteam, Scottgames]",3.0,177,Lists,"[Adventure, Indie, Point-and-Click, Simulator,...","Welcome to Circus Baby's Pizza World, where fa...","[minus 2 for night 4, So We All Universally Ag...",2.1K,9,228,123
117,Friday the 13th: The Game,"May 25, 2017","[Illfonic, Gun Interactive]",2.6,181,Lists,[Indie],Friday the 13th: The Game is a third-person ho...,[this game was great fun. 2 stars because the ...,2.3K,12,204,74
118,SteamWorld Dig 2,"Sep 21, 2017",[Image & Form],3.8,324,Lists,"[Adventure, Indie, Platform, Puzzle]",SteamWorld Dig 2 takes you on a platform minin...,[Steamworld Dig 2 is a polished gem in both a ...,1.8K,35,616,179
119,Dark Souls: Artorias of the Abyss,"Aug 24, 2012","[FromSoftware, Namco Bandai Games]",4.4,117,Lists,[RPG],Artorias of the Abyss is additional content fo...,"[Dark souls 1 alcança niveis ainda maiores...,...",1.9K,5,141,58


In [ ]:
backloggd.to_csv('backloggd.csv')